In [1]:
# ! pip install sentencepiece

In [2]:
import torch
print("PyTorch:", torch.__version__)
print("MPS built:", torch.backends.mps.is_built())
print("MPS available:", torch.backends.mps.is_available())

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(device)

import torch.nn as nn
import math

PyTorch: 2.9.1
MPS built: True
MPS available: True
mps


### hyperparameters

In [3]:
vocab_size=800
d_model=256
max_seq_len = 128
num_heads=8
num_encoder_layers=6
num_decoder_layers=6
d_ff=2048

### files

In [4]:
from datasets import load_dataset
from tqdm.auto import tqdm

def create_data_files(source_file, target_file,dataset,max_line_count=None):
    line_count = 0
    for example in tqdm(dataset):
        source_text = example["source"].replace("\n", " ").strip()
        target_text = example["translated"].replace("\n", " ").strip()
        source_file.write(source_text + "\n")
        target_file.write(target_text + "\n")
        line_count=line_count+1
        if max_line_count is not None:
            if line_count >= max_line_count:
                break

    source_file.close()
    target_file.close()

/Users/mytkn/PycharmProjects/SmallLanguageModels/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
check_file = True
if check_file:
    ds = load_dataset("cturan/high-quality-english-turkish-sentences")

    train_line_count = None

    train_source_file = open("data/train_source.txt", "w")
    train_target_file = open("data/train_target.txt", "w")

    create_data_files(train_source_file, train_target_file,ds["train"],train_line_count)

In [6]:
import os

def delete_files_in_folder(folder_path):
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        try:
            if os.path.isfile(file_path):
                os.remove(file_path)
                print(f'Deleted: {file_path}')
        except Exception as e:
            print(f'Error: {e}')

delete_files_in_folder('tokenizers')

Deleted: tokenizers/tr_bpe_32k.model
Deleted: tokenizers/en_bpe_32k.vocab
Deleted: tokenizers/en_bpe_32k.model
Deleted: tokenizers/tr_bpe_32k.vocab


In [7]:
def print_test_heading(heading):
    print("-"*60)
    print(heading)
    print()

In [8]:
import sentencepiece as spm
import os

# ============================================================
# TÜRKÇE TOKENİZER EĞİTİMİ
# ============================================================

def train_tokenizer(input_file, model_prefix, vocab_size=32000):
    """
    SentencePiece BPE tokenizer eğit.

    Args:
        input_file: Eğitim corpus dosyası (her satır bir cümle)
        model_prefix: Çıktı model ismi (örn: 'tr_bpe_32k')
        vocab_size: Vocabulary boyutu
    """
    print(f"✅ Tokenizer eğitimi başladı: {model_prefix}.model")

    spm.SentencePieceTrainer.train(
        input=input_file,
        model_prefix=model_prefix,
        vocab_size=vocab_size,
        model_type='bpe',

        # Özel tokenler
        pad_id=0,
        unk_id=1,
        bos_id=2,
        eos_id=3,
        pad_piece='<PAD>',
        unk_piece='<UNK>',
        bos_piece='<BOS>',
        eos_piece='<EOS>',

        # Eğitim parametreleri
        character_coverage=0.9995,  # Türkçe karakterler için yüksek
        num_threads=32,
        train_extremely_large_corpus=False,

        # Normalizasyon
        normalization_rule_name='nmt_nfkc_cf',  # Unicode normalizasyonu
        remove_extra_whitespaces=True,


        # Subword regularization (opsiyonel, eğitimde çeşitlilik için)
        # split_by_unicode_script=True,
        # byte_fallback=True,  # Bilinmeyen karakterler için
    )

    print(f"✅ Tokenizer eğitildi: {model_prefix}.model")

# Corpus oluştur ve tokenizer'ları eğit
tr_corpus, en_corpus =  "data/train_target.txt", "data/train_source.txt"

# NOT: Gerçek projede vocab_size=32000, demo için 1000
train_tokenizer(tr_corpus, 'tokenizers/tr_bpe_32k', vocab_size=vocab_size)
train_tokenizer(en_corpus, 'tokenizers/en_bpe_32k', vocab_size=vocab_size)

print("\n✅ Her iki tokenizer da hazır!")

✅ Tokenizer eğitimi başladı: tokenizers/tr_bpe_32k.model
✅ Tokenizer eğitildi: tokenizers/tr_bpe_32k.model
✅ Tokenizer eğitimi başladı: tokenizers/en_bpe_32k.model
✅ Tokenizer eğitildi: tokenizers/en_bpe_32k.model

✅ Her iki tokenizer da hazır!


sentencepiece_trainer.cc(78) LOG(INFO) Starts training with : 
trainer_spec {
  input: data/train_target.txt
  input_format: 
  model_prefix: tokenizers/tr_bpe_32k
  model_type: BPE
  vocab_size: 800
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 32
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  seed_sentencepieces_file: 
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 1
  bos_id: 2
  eos_id: 3
  pad_id: 0
  unk_piece: <UNK>
  bos_piece: <BOS>
  eos_piece: <EOS>
  pad_piece: <PAD>
  unk_surface:  ⁇ 
  enable_differential

In [9]:
from tokenizer import SentencePieceTokenizer

tr_tokenizer = SentencePieceTokenizer('tokenizers/tr_bpe_32k.model')
en_tokenizer = SentencePieceTokenizer('tokenizers/en_bpe_32k.model')


# test
print_test_heading("SentencePiece Test")
tr_tokens = tr_tokenizer.encode("Merhaba dünya")
print(tr_tokens)
print(tr_tokenizer.decode(tr_tokens))

en_tokens = en_tokenizer.encode("Hello world")
print(en_tokens)
print(en_tokenizer.decode(en_tokens))

------------------------------------------------------------
SentencePiece Test

[2, 589, 769, 258, 747, 668, 3]
merhaba dünya
[2, 108, 281, 754, 482, 3]
hello world


### token embedding

In [10]:
class TokenEmbedding(nn.Module):

    """
    kelimeleri sayısal vektörlere dönüştürmek için kullanılan bir katman.
    """

    def __init__(self,vocab_size:int,d_model:int):
        """
        :param vocab_size: sözlükteki toplam kelime sayisi
        :param d_model: her kelimenin vektör boyutu
        """
        super(TokenEmbedding,self).__init__()
        self.d_model=d_model
        self.vocab_size=vocab_size
        self.embedding=nn.Embedding(self.vocab_size,self.d_model)


    def forward(self,x):
        """
        :param x: token tensorü (batch_size, seq_len)
        :return: embedding tensorü (batch_size, seq_len, d_model)
        """
        return self.embedding(x)*math.sqrt(self.d_model)

### positional encoding

In [11]:
class PositionalEncoding(nn.Module):
    """
    Kelimelerin cümle içindeki konum bilgilerini eklemek için kullanılan bir katman.
    """

    def __init__(self,d_model:int,max_seq_len:int=500,dropout:float=0.1):
        """
        :param d_model: embedding boyutu
        :param max_seq_len: max cümle uzunluğu
        :param dropout: regularizasyon için dropout oranı
        """
        super(PositionalEncoding,self).__init__()
        self.d_model=d_model
        self.max_seq_len=max_seq_len
        self.dropout=nn.Dropout(dropout)
        # pozisyonel encoding matrisini oluştur
        pe=torch.zeros(self.max_seq_len,self.d_model)
        position=torch.arange(0,self.max_seq_len).unsqueeze(1).float()

        div_term=torch.exp(torch.arange(0,self.d_model,2).float()*(-math.log(10000.0)/self.d_model))

        pe[:,0::2]=torch.sin(position*div_term) # çift indexler sin
        pe[:,1::2]=torch.cos(position*div_term) # tek indexler cos

        pe=pe.unsqueeze(0) # (1, max_seq_len, d_model)

        # model ile kaydet, fakat gradient hesaplanmasın
        self.register_buffer('pe',pe)

    def forward(self,x):
        """
        :param x: token embedding tensorü (batch_size, seq_len, d_model)
        :return: position-encoded embedding tensorü
        """
        seq_len = x.size(1)
        x = x + self.pe[:, :seq_len, :]
        return self.dropout(x)

### sclaed Dot-Product attention

In [12]:
class ScaledDotProductAttention(nn.Module):
    """
    Attention mekanizması, modelin önemli bilgilere odaklanmasını sağlar.
    FORMÜL:
    ───────
    Attention(Q, K, V) = softmax(Q·Kᵀ / √d_k) · V

    - Q (Query): "Ne arıyorum?"
    - K (Key): "Ben neyim?"
    - V (Value): "Değerim ne?"

    √d_k ile bölme: Skorların çok büyümesini önler (softmax'ı stabilize eder)
    """

    def __init__(self,dropout:float=0.1):
        """
        :param dropout: regularizasyon için dropout oranı
        """
        super(ScaledDotProductAttention,self).__init__()
        self.dropout=nn.Dropout(dropout)

    def forward(self,query,key,value,mask=None):
        """
        :param query: [batch_size, num_heads, seq_len, d_k]
        :param key: [batch_size, num_heads, seq_len, d_k]
        :param value: [batch_size, num_heads, seq_len, d_k]
        :param mask: opsiyonel maske tensorü

        :return: output :Attention uygulanmış tensor
        :return :attention_weights : dikkat ağırlıkları, her pozisyonun diğerlerine verdiği önem
        """
        d_k = query.size(-1)  # d_k boyutu
        # Q·Kᵀ hesapla
        scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k)

        if mask is not None:
            scores = scores.masked_fill(mask == 0, float('-inf'))

        attention_weights = torch.softmax(scores, dim=-1)  # softmax uygula
        attention_weights = self.dropout(attention_weights)  # dropout uygula

        # softmax sonrası dropout yapınca, satırların toplamı 1 olmayabilir, normalize edelim
        temp_attention_weights = attention_weights.sum(dim=-1, keepdim=True).clamp(min=1e-9)
        attention_weights = attention_weights / temp_attention_weights

        output = torch.matmul(attention_weights, value)
        return output,attention_weights


In [13]:
print_test_heading("Scaled Dot-Product Attention Test")
attention = ScaledDotProductAttention(dropout=0.1)
batch_size, num_heads, seq_len, d_k = 1, 1, 4, 64
Q = torch.randn(batch_size, num_heads, seq_len, d_k)
K = torch.randn(batch_size, num_heads, seq_len, d_k)
V = torch.randn(batch_size, num_heads, seq_len, d_k)

output, attn_weights = attention(Q, K, V)

print(f"Query shape: {Q.shape}")
print(f"Attention output shape: {output.shape}")
print(f"Attention weights shape: {attn_weights.shape}")
print(f"\nAttention weights (her kelime diğerlerine ne kadar bakıyor):")
print(attn_weights.squeeze())
print(f"\nHer satırın toplamı (1 olmalı): {attn_weights.squeeze().sum(dim=-1)}")

------------------------------------------------------------
Scaled Dot-Product Attention Test

Query shape: torch.Size([1, 1, 4, 64])
Attention output shape: torch.Size([1, 1, 4, 64])
Attention weights shape: torch.Size([1, 1, 4, 4])

Attention weights (her kelime diğerlerine ne kadar bakıyor):
tensor([[0.0000, 0.8948, 0.1052, 0.0000],
        [0.2312, 0.1593, 0.5557, 0.0538],
        [0.0492, 0.6531, 0.1307, 0.1671],
        [0.1505, 0.0000, 0.2174, 0.6321]])

Her satırın toplamı (1 olmalı): tensor([1., 1., 1., 1.])


### multi-head attention

In [14]:
class MultiHeadAttention(nn.Module):
    """
    birden fazla attention "kafası" ile aynı anda farklı bilgilere odaklanmayı sağlar.

    Cümle: "Bankaya gittim para çekmek için"

    Farklı head'ler farklı şeylere odaklanır:
    - Head 1: Sözdizimi ilişkisi → "gittim" ← "için" (amaç ilişkisi)
    - Head 2: Anlamsal ilişki → "banka" ← "para" (finans bağlamı)
    - Head 3: Özne-fiil ilişkisi → "gittim" ← (gizli ben)
    - Head 4: Nesne ilişkisi → "çekmek" ← "para"

    Tek head tüm bu ilişkileri aynı anda yakalayamaz!
    Multi-head ile paralel olarak farklı pattern'ler öğrenilir.

    FORMÜL:
    ───────
    MultiHead(Q, K, V) = Concat(head_1, ..., head_h) · W_O
    head_i = Attention(Q·W_Q_i, K·W_K_i, V·W_V_i)
    """

    def __init__(self,d_model:int,num_heads:int,dropout:float=0.1):
        """
        :param d_model: model boyutu boyutu
        :param num_heads: attention kafası sayısı
        :param dropout: regularizasyon için dropout oranı
        """
        super(MultiHeadAttention,self).__init__()

        assert d_model % num_heads == 0 , "d_model, num_heads ile tam bölünmeli"

        self.d_model=d_model
        self.num_heads=num_heads
        self.d_k = d_model // num_heads # her head'in boyutu

        # Q, K, V ve çıktı için lineer dönüşümler
        self.W_q = nn.Linear(d_model,d_model)
        self.W_k = nn.Linear(d_model,d_model)
        self.W_v = nn.Linear(d_model,d_model)
        self.W_o = nn.Linear(d_model,d_model)

        self.attention = ScaledDotProductAttention(dropout)
        self.dropout = nn.Dropout(dropout)

    def forward(self,query,key,value,mask=None):
        """
        :param query: [batch_size, seq_len, d_model]
        :param key: [batch_size, seq_len, d_model]
        :param value: [batch_size, seq_len, d_model]
        :param mask: opsiyonel maske tensorü

        :return: output :Multi-head attention uygulanmış tensor [batch_size, seq_len, d_model]
        :return: attention_weights : dikkat ağırlıkları [batch_size, num_heads, seq_len, seq_len]
        """
        batch_size = query.size(0)

        # linear projeksiyonlar
        Q = self.W_q(query)  # (batch_size, seq_len, d_model)
        K = self.W_k(key)    # (batch_size, seq_len, d_model)
        V = self.W_v(value)  # (batch_size, seq_len, d_model)

        # headlere böl
        Q = Q.reshape(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)  # (batch, num_heads, seq_len, d_k)
        K = K.reshape(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        V = V.reshape(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)

        # attention uygula
        attn_output, attention_weights = self.attention(Q, K, V, mask)  # (batch_size, num_heads, seq_len, d_k)

        # headleri birleştir
        # [batch, num_heads, seq_len, d_k] → [batch, seq_len, d_model]
        attn_output = attn_output.transpose(1, 2).contiguous()
        attn_output = attn_output.view(batch_size, -1, self.d_model)

        # final lineer dönüşüm
        output = self.W_o(attn_output)  # (batch_size, seq_len, d_model)
        output = self.dropout(output)

        return output, attention_weights

### position-wise feed-forward network

In [15]:
class PositionwiseFeedForward(nn.Module):
    """
    her pozisyon için ayrı ayrı uygulanan iki katmanlı tam bağlantılı sinir ağı.
    Attention -> hangi kelimeye bakmalıyım? sorusunu cevaplar
    Feed-Forward -> o kelimeden ne öğrenmeliyim? sorusunu cevaplar

    her kelime vektörü için:
    - Genişletme (expansion): Boyutu artırarak daha karmaşık özellikler öğrenilir.
    - Aktivasyon: Non-lineerlik eklenir (ReLU gibi).
    - Daraltma (projection): Orijinal boyuta geri döner.

    FFN(x) = ReLU(x·W₁ + b₁)·W₂ + b₂
    """
    def __init__(self,d_model,d_ff,dropout:float=0.1):
        """
        :param d_model: model boyutu
        :param d_ff: feed-forward katmanının ara boyutu
        :param dropout: regularizasyon için dropout oranı
        """
        super(PositionwiseFeedForward,self).__init__()
        self.linear1 = nn.Linear(d_model,d_ff)
        self.linear2 = nn.Linear(d_ff,d_model)
        self.dropout = nn.Dropout(dropout)
        self.activation = nn.ReLU()

    def forward(self,x):
        """
        :param x: input tensorü (batch_size, seq_len, d_model)
        :return: output tensorü (batch_size, seq_len, d_model)
        """
        x = self.linear1(x)
        x = self.activation(x)
        x = self.dropout(x)
        x = self.linear2(x)
        return x


### layer normalization

In [16]:
class AddAndNorm(nn.Module):
    """
    Residual bağlantı + layer normalization

    Residual => vanishing gradient problemini azaltır ve eğitim stabilitesini artırır.
    Layer Norm => her katmanda aktivasyonları normalize eder, eğitim hızını artırır.
    """
    def __init__(self,d_model,dropout:float=0.1):
        super(AddAndNorm,self).__init__()
        self.layer_norm=nn.LayerNorm(d_model)
        self.dropout=nn.Dropout(dropout)

    def forward(self,x,sublayer_output):
        """
        :param x: orjinal girdi tensorü
        :param sublayer_output: alt katman çıktısı tensorü (attention veya feed-forward)
        :return: normalize edilmiş çıktı tensorü
        """
        return self.layer_norm(x + self.dropout(sublayer_output))


class PreNormAndNorm(nn.Module):
    """
    Pre-Layer Normalization + Residual bağlantı + Layer Normalization
    """

    def __init__(self,d_model,dropout:float=0.1):
        super(PreNormAndNorm,self).__init__()
        self.layer_norm=nn.LayerNorm(d_model)
        self.dropout=nn.Dropout(dropout)

    def forward(self,x,sublayer):
        """
        Pre-LN: Norm -> Sublayer -> Add
        """
        normalized = self.layer_norm(x)
        return x + self.dropout(sublayer(normalized))


### encoder layer

In [17]:
class EncoderLayer(nn.Module):
    """
    tek bir encoder katmanı : Self-attention + Feed-Forward

    örnek : Kedi sütü içti.
    Encoder layer -> kelimeler birbirlerine bakar..
        içti -> kediye bakar (özne-fiil ilişkisi)
        içti -> süt'e bakar (nesne ilişkisi)
    sonra her kelime için ne öğrenmesi gerektiğine karar verir.
    Feed forward -> her kelime için zengin temsil oluşturur
        içti : kedi + süt + geçmiş zaman bilgisi
    """

    def __init__(self,d_model:int,num_heads:int,d_ff:int,dropout:float=0.1):
        """
        :param d_model: model boyutu
        :param num_heads: attention kafası sayısı
        :param d_ff: feed-forward katmanının ara boyutu
        :param dropout: regularizasyon için dropout oranı
        """
        super(EncoderLayer,self).__init__()
        # sublayer 1
        self.self_attention=MultiHeadAttention(d_model,num_heads,dropout)
        self.norm1=nn.LayerNorm(d_model)

        #sublayer 2
        self.feed_forward=PositionwiseFeedForward(d_model,d_ff,dropout)
        self.norm2=nn.LayerNorm(d_model)

        self.dropout=nn.Dropout(dropout)

    def forward(self,x,src_mask=None):
        """
        :param x: input tensorü [batch_size, seq_len, d_model]
        :param src_mask: opsiyonel kaynak maske tensorü

        :return: output tensorü [batch_size, seq_len, d_model]
        """
        # Sublayer 1: Self-Attention with residual and Layer Norm
        attn_output, _ = self.self_attention(x, x, x, src_mask)
        x = self.norm1(x + self.dropout(attn_output))

        # Sublayer 2: Feed-Forward with residual and Layer Norm
        ffn_output = self.feed_forward(x)
        x = self.norm2(x + self.dropout(ffn_output))

        return x

### decoder layer

In [18]:
class DecoderLayer(nn.Module):
    """
    tek bir decoder katmanı : Masked Self-attention + Cross-Attention + Feed-Forward
    örnek : Kedi sütü içti. -> The cat drank the milk.

    Decoder "the cat" üretmiş, şimdi "drank" üretmeli.
    - Masked Self-Attention -> sadece "the" ve "cat" kelimelerine bakabilir. milk e bakamaz, o gelecekte.
    - Cross-Attention -> Türkçe encoder çıktısın bakar, "içti" kelimesine yüksek attention -> "drank"
    - Feed-Forward -> tüm bilgiyi işle ve final temsil oluştur
    """
    def __init__(self,d_model:int,num_heads:int,d_ff:int,dropout:float=0.1):
        super(DecoderLayer, self).__init__()
        # sublayer 1: masked multi-head self-attention
        self.masked_self_attention=MultiHeadAttention(d_model,num_heads,dropout)
        self.norm1=nn.LayerNorm(d_model)

        # sublayer 2: multi-head cross-attention (encoder-decoder attention)
        self.cross_attention=MultiHeadAttention(d_model,num_heads,dropout)
        self.norm2=nn.LayerNorm(d_model)

        #sublayer 3: position-wise feed-forward
        self.feed_forward=PositionwiseFeedForward(d_model,d_ff,dropout)
        self.norm3=nn.LayerNorm(d_model)

        self.dropout=nn.Dropout(dropout)

    def forward(self,x,encoder_output, src_mask=None,tgt_mask=None):
        """
        :param x: decoder input tensorü [batch_size, tgt_seq_len, d_model]
        :param encoder_output: encoder çıktısı tensorü [batch_size, src_seq_len, d_model]
        :param src_mask: source padding mask tensorü
        :param tgt_mask: target casual mask tensorü (look-ahead + padding) gelecek kelimelere bakmamak için
        :return: [batch_size, tgt_seq_len, d_model]
        """
        # Sublayer 1: Masked Self-Attention
        # Q=K=V=decoder input, mask ile gelecek görünmez
        attn_output, _ = self.masked_self_attention(x, x,x, tgt_mask)
        x = self.norm1(x + self.dropout(attn_output))

        #sublayer 2: Cross-Attention
        # Q=decoder output, K=V=encoder output
        attn_output, _ = self.cross_attention(x, encoder_output, encoder_output, src_mask)
        x = self.norm2(x + self.dropout(attn_output))

        #sublayer 3: Feed-Forward
        ffn_output = self.feed_forward(x)
        x = self.norm3(x + self.dropout(ffn_output))
        return x


### full encoder

In [19]:
class Encoder(nn.Module):
    """
    n tane encoder katmanından oluşan tam encoder.

    örnek : bugün hava çok güzel
    layer 1 :temel ilişkiler -> güzel -> hava (sıfat-isim ilişkisi)
    layer 2: kompozit anlam : güzel hava -> kavram
    layer 3: bağlam bilgisi : bugün güzel hava (zaman bilgisi, bugüne özgü durum)
    layer 4-6: daha soyut ilişkiler ve anlamlar öğrenir. duygu, niyet vb.
    daha fazla katman -> daha derin anlamlar öğrenme, daha fazla parametre, daha fazla hesaplama
    """
    def __init__(
        self,
        vocab_size:int,
        d_model:int,
        num_heads:int,
        num_layers:int,
        d_ff:int,
        max_seq_len:int,
        dropout:float=0.1
    ):
        super(Encoder,self).__init__()

        # token embedding + positional encoding
        self.token_embedding=TokenEmbedding(vocab_size,d_model)
        self.positional_encoding=PositionalEncoding(d_model,max_seq_len,dropout)

        # n adet encoder layer
        self.layers = nn.ModuleList([
            EncoderLayer(d_model,num_heads,d_ff,dropout)
            for _ in range(num_layers)
        ])

        self.norm = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self,src,src_mask=None):
        """
        :param src: kaynak cümle tensorü [batch_size, src_seq_len] source token ids
        :param src_mask: opsiyonel kaynak maske tensorü
        :return: encoder çıktısı tensorü [batch_size, src_seq_len, d_model
        """
        # token embedding
        x = self.token_embedding(src)  # (batch_size, src_seq_len, d_model)

        # poisitional encoding
        x = self.positional_encoding(x)  # pozisyon bilgisi ekle

        # dropout uygula
        x = self.dropout(x)

        # n encoder layer'dan geçir
        for layer in self.layers:
            x = layer(x, src_mask)

        # final normalization
        return self.norm(x)

### decoder

In [20]:
class Decoder(nn.Module):
    """
    N adet decoder layer'dan oluşan tam decoder.

    Encoder: "Kedi süt içti" → [encoded_context]

    Decoder adım adım çeviri üretir:

    Adım 1: <BOS> → "The"
    Adım 2: <BOS> The → "cat"
    Adım 3: <BOS> The cat → "drank"
    Adım 4: <BOS> The cat drank → "milk"
    Adım 5: <BOS> The cat drank milk → <EOS>

    Her adımda:
    1. Şu ana kadar üretilenlere bak (masked self-attention)
    2. Encoder'a bak, Türkçe ne diyordu? (cross-attention)
    3. Sonraki kelimeyi tahmin et
    """

    def __init__(
            self,
            vocab_size:int,
            d_model:int,
            num_heads:int,
            num_layers:int,
            d_ff:int,
            max_seq_len:int,
            dropout:float=0.1
    ):
        super(Decoder,self).__init__()

        self.token_embedding=TokenEmbedding(vocab_size,d_model)
        self.positional_encoding=PositionalEncoding(d_model,max_seq_len,dropout)

        self.layers = nn.ModuleList([
            DecoderLayer(d_model,num_heads,d_ff,dropout)
            for _ in range(num_layers)
        ])

        self.norm = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self,tgt,encoder_output, src_mask=None,tgt_mask=None):
        """
        :param tgt: hedef cümle tensorü [batch_size, tgt_seq_len] target token ids
        :param encoder_output: encoder çıktısı tensorü [batch_size, src_seq_len,
        d_model]
        :param src_mask: kaynak maske tensorü
        :param tgt_mask: hedef maske tensorü
        :return: decoder çıktısı tensorü [batch_size, tgt_seq_len, d_model]
        """
        x = self.token_embedding(tgt)
        x = self.positional_encoding(x)

        # dropout uygula
        x = self.dropout(x)

         # n decoder layer'dan geçir
        for layer in self.layers:
            x = layer(x,encoder_output,src_mask,tgt_mask)

        return self.norm(x)


### complete transformer model

In [21]:
class Transformer(nn.Module):
    """
    Türkçe-İngilizce çeviri için tam Transformer modeli.
    Akış:
    Cümle : Merhaba dünya
    Tokenizer - > Token IDs : [45, 678, 23]
    Encoder -> Türkçenin zengin temsili
    Decoder -> <BOS> -> Hello -> world -> <EOS>
    Cümle : Hello world
    """
    def __init__(
            self,
            src_vocab_size:int, #türkçe sözlük boyutu
            tgt_vocab_size:int, #ingilizce sözlük boyutu
            d_model:int, #model boyutu
            num_heads:int, #attention kafası sayısı
            num_encoder_layers:int, #encoder katman sayısı
            num_decoder_layers:int, #decoder katman sayısı
            d_ff:int, #feed-forward ara boyutu
            max_seq_len:int, # max cümle uzunluğu
            dropout:float=0.1,
            padd_idx:int=0 #padding token id
    ):
        super(Transformer,self).__init__()

        self.pad_idx=padd_idx
        self.d_model=d_model

        #encoder
        self.encoder=Encoder(
            src_vocab_size,
            d_model,
            num_heads,
            num_encoder_layers,
            d_ff,
            max_seq_len,
            dropout
        )

        #decoder
        self.decoder=Decoder(
            tgt_vocab_size,
            d_model,
            num_heads,
            num_decoder_layers,
            d_ff,
            max_seq_len,
            dropout
        )

        # final output projeksiyonu : d_model -> tgt_vocab_size
        # her pozisyon için kelime olasılıkları
        self.output_projections=nn.Linear(d_model,tgt_vocab_size)

        self._init_parameters()

    def _init_parameters(self):
        """
        model parametrelerini Xavier uniform ile başlat
        """
        for p in self.parameters():
            if p.dim()>1:
                nn.init.xavier_uniform_(p)

    def make_src_mask(self,src):
        """
        kaynak cümle için padding maskesi oluştur
        :param src: kaynak cümle tensorü [batch_size, src_seq_len]
        :return: src_mask: [batch_size, 1, 1, src_seq_len]
        """
        src_mask = (src != self.pad_idx).unsqueeze(1).unsqueeze(2)
        return src_mask  # (batch_size, 1, 1, src_seq_len)

    def make_tgt_mask(self,tgt):
        """
        hedef cümle için casual (look-ahead + padding) maske oluştur
        :param tgt: hedef cümle tensorü [batch_size, tgt_seq_len]
        :return: tgt_mask: [batch_size, 1, tgt_seq_len, tgt_seq_len]
        """

        batch_size, tgt_len = tgt.shape

        # Padding mask
        tgt_pad_mask = (tgt != self.pad_idx).unsqueeze(1).unsqueeze(2)
        # [batch, 1, 1, tgt_len]

        # Causal mask (üst üçgen sıfır)
        causal_mask = torch.tril(torch.ones(tgt_len, tgt_len, device=tgt.device))
        causal_mask = causal_mask.unsqueeze(0).unsqueeze(1)
        # [1, 1, tgt_len, tgt_len]

        # İkisini birleştir
        tgt_mask = tgt_pad_mask & causal_mask.bool()
        return tgt_mask

    def forward(self,src,tgt):
        """
        :param src: en - kaynak cümle tensorü [batch_size, src_seq_len]
        :param tgt: tr - hedef cümle tensorü [batch_size, tgt_seq_len]
        :return: output logits tensorü [batch_size, tgt_seq_len, tgt_vocab_size]
        """
        src_mask = self.make_src_mask(src)
        tgt_mask = self.make_tgt_mask(tgt)

        encoder_output = self.encoder(src, src_mask)

        decoder_output = self.decoder(tgt, encoder_output, src_mask, tgt_mask)

        logits = self.output_projections(decoder_output)

        return logits

    def encode(self,src,src_mask=None):
        if src_mask is None:
            src_mask = self.make_src_mask(src)
        return self.encoder(src, src_mask)

    def decode(self, tgt, encoder_output, src_mask=None, tgt_mask=None):
        if tgt_mask is None:
            tgt_mask = self.make_tgt_mask(tgt)
        decoder_output = self.decoder(tgt, encoder_output, src_mask, tgt_mask)
        return self.output_projections(decoder_output)

### training loop

In [22]:
def read_corpus_lines(corpus_path, encoding='utf-8', strip=True, skip_empty=True):
    """
    Dosyayı satır satır okur ve temizlenmiş satırları liste olarak döner.
    Args:
      - corpus_path: path string (ör. `data/train_target.txt`)
      - encoding: dosya encoding'i
      - strip: her satırı strip() ile temizle
      - skip_empty: boş satırları atla
    Returns:
      - list of str
    """
    lines = []
    with open(corpus_path, 'r', encoding=encoding) as f:
        for ln in f:
            s = ln.rstrip('\n')
            if strip:
                s = s.strip()
            if skip_empty and not s:
                continue
            lines.append(s)
    return lines

def get_corpus_and_tokenizers():
    """
    Eğitim corpuslarını ve tokenizer'ları yükle.
    Returns:
        tr_corpus: Türkçe eğitim corpus dosyası
        en_corpus: İngilizce eğitim corpus dosyası
        tr_tokenizer: Türkçe SentencePiece tokenizer
        en_tokenizer: İngilizce SentencePiece tokenizer
    """
    tr_corpus = "data/train_target.txt"
    en_corpus = "data/train_source.txt"

    tr_tokenizer = SentencePieceTokenizer('tokenizers/tr_bpe_32k.model')
    en_tokenizer = SentencePieceTokenizer('tokenizers/en_bpe_32k.model')

    return tr_corpus, en_corpus, tr_tokenizer, en_tokenizer

In [23]:
tr_corpus, en_corpus, tr_tokenizer, en_tokenizer = get_corpus_and_tokenizers()

tr_sentences = read_corpus_lines(tr_corpus)
en_sentences = read_corpus_lines(en_corpus)


In [24]:
print(tr_sentences[0])
print(en_sentences[0])

Çalışmalarının çoğu N.C. Eğitim Araştırma Veri Merkezine dayanmaktadır.
Many of their studies rely on the N.C. Education Research Data Center.


In [25]:
from torch.utils.data import DataLoader, TensorDataset, random_split

def create_dataloader(src_encoded,tgt_encoded,batch_size=16,val_split=0.1):
    dataset = TensorDataset(src_encoded,tgt_encoded)
    val_size = int(len(dataset)*val_split)
    train_size = len(dataset) - val_size

    train_dataset, val_dataset = random_split(dataset,[train_size,val_size])

    train_loader = DataLoader(train_dataset,batch_size=batch_size,shuffle=True)
    val_loader = DataLoader(val_dataset,batch_size=batch_size,shuffle=False)
    return train_loader, val_loader

In [26]:
import torch.optim as optim
from torch.utils.data import DataLoader,Dataset

class TranslationDataSet(Dataset):
    """
    çeviri veri seti için özel Dataset sınıfı.
    """
    def __init__(self,src_sentences,tgt_sentences,src_tokenizer,tgt_tokenizer,max_len=max_seq_len):
        """
        :param src_sentences: kaynak cümleler listesi
        :param tgt_sentences: hedef cümleler listesi
        :param src_tokenizer: kaynak tokenizer
        :param tgt_tokenizer: hedef tokenizer
        :param max_len: max cümle uzunluğu
        """
        self.src_sentences=src_sentences
        self.tgt_sentences=tgt_sentences
        self.src_tokenizer=src_tokenizer
        self.tgt_tokenizer=tgt_tokenizer
        self.max_len=max_len

    def __len__(self):
        return len(self.src_sentences)

    def __getitem__(self,idx):
        src = self.src_tokenizer.encode(self.src_sentences[idx])
        tgt = self.tgt_tokenizer.encode(self.tgt_sentences[idx])

        # max_len'e göre padding/truncation
        src = src[:self.max_len]
        src = src + [0] * max(0, self.max_len - len(src))

        tgt = tgt[:self.max_len]
        tgt = tgt + [0] * max(0, self.max_len - len(tgt))

        return torch.tensor(src , dtype=torch.long), torch.tensor(tgt, dtype=torch.long)

In [27]:
class LabelSmoothingLoss(nn.Module):
    """
    Label Smoothing: Overconfidence'ı önler.

    Normal: target = [0, 0, 1, 0, 0] (kesin "cat")
    Smoothed: target = [0.02, 0.02, 0.92, 0.02, 0.02]
    """

    def __init__(self, vocab_size: int, pad_idx: int = 0, smoothing: float = 0.1):
        super().__init__()
        self.vocab_size = vocab_size
        self.pad_idx = pad_idx
        self.smoothing = smoothing
        self.confidence = 1.0 - smoothing

    def forward(self, logits, target):
        """
        Args:
            logits: [batch * seq_len, vocab_size]
            target: [batch * seq_len]
        """
        logits = logits.reshape(-1, self.vocab_size)
        target = target.reshape(-1)

        # Smooth distribution
        smooth_target = torch.zeros_like(logits)
        smooth_target.fill_(self.smoothing / (self.vocab_size - 2))
        smooth_target.scatter_(1, target.unsqueeze(1), self.confidence)
        smooth_target[:, self.pad_idx] = 0

        # Padding mask
        mask = (target != self.pad_idx)

        # Cross entropy with smooth targets
        log_probs = torch.log_softmax(logits, dim=-1)
        loss = -smooth_target * log_probs
        loss = loss.sum(dim=-1)
        loss = loss.masked_select(mask).mean()

        return loss

In [28]:
def train_epoch(model, dataloader, optimizer, criterion, device):
    """Bir epoch eğitim."""
    model.train()
    total_loss = 0

    for batch_idx, (src, tgt) in enumerate(dataloader):
        src = src.to(device)
        tgt = tgt.to(device)

        # print("SHAPE",src.shape, tgt.shape)

        # Decoder input: <BOS> + target[:-1]
        tgt_input = tgt[:, :-1]
        # Decoder target: target[1:] + <EOS>
        tgt_output = tgt[:, 1:]

        # Forward pass
        optimizer.zero_grad()
        logits = model(src, tgt_input)

        # Loss hesapla
        loss = criterion(logits, tgt_output)

        # Backward pass
        loss.backward()

        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        optimizer.step()
        total_loss += loss.item()

        if batch_idx % batch_size == 0:
            print(f"  Batch {batch_idx}, Loss: {loss.item():.4f}")

    return total_loss / len(dataloader)

In [29]:
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts

print_test_heading("Training Loop")


model = Transformer(
    src_vocab_size=vocab_size,
    tgt_vocab_size=vocab_size,
    d_model=d_model,
    num_heads=num_heads,
    num_encoder_layers=num_encoder_layers,
    num_decoder_layers=num_decoder_layers,
    d_ff=d_ff,
    max_seq_len=max_seq_len,
    dropout=0.1,
    padd_idx=0
).to(device)

optimizer = optim.AdamW(
    model.parameters(),
    lr=1e-4,
    betas=(0.9, 0.98),
    eps=1e-9,
    weight_decay=0.01
)

criterion = LabelSmoothingLoss(vocab_size=vocab_size, pad_idx=0, smoothing=0.1)

print(f"\nModel hazır!")
print(f"Parametre sayısı: {sum(p.numel() for p in model.parameters()):,}")



num_epochs = 3
batch_size = 32

dataset=TranslationDataSet(en_sentences,tr_sentences,en_tokenizer,tr_tokenizer)

dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, drop_last=False)

scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=2)

from datetime import datetime

start = datetime.now()

for epoch in range(num_epochs):
    loss = train_epoch(model, dataloader, optimizer, criterion, device)

    scheduler.step()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss:.4f}")

end = datetime.now()
print(f"\nEğitim süresi: {(end - start).total_seconds() / 60} dakika")


------------------------------------------------------------
Training Loop


Model hazır!
Parametre sayısı: 17,979,168
  Batch 0, Loss: 6.8854
  Batch 32, Loss: 6.2551
  Batch 64, Loss: 6.1708
  Batch 96, Loss: 6.2009
  Batch 128, Loss: 6.1361
  Batch 160, Loss: 6.1414
  Batch 192, Loss: 6.1097
  Batch 224, Loss: 6.0539
  Batch 256, Loss: 6.0825
  Batch 288, Loss: 5.9839
  Batch 320, Loss: 6.0479
  Batch 352, Loss: 6.0054
  Batch 384, Loss: 6.0168
  Batch 416, Loss: 5.9650
  Batch 448, Loss: 6.0141
  Batch 480, Loss: 5.9458
  Batch 512, Loss: 5.9948
  Batch 544, Loss: 5.8750
  Batch 576, Loss: 6.0091
  Batch 608, Loss: 5.9095
  Batch 640, Loss: 5.8913
  Batch 672, Loss: 5.8207
  Batch 704, Loss: 5.7467
  Batch 736, Loss: 5.7972
  Batch 768, Loss: 5.7188
  Batch 800, Loss: 5.6573
  Batch 832, Loss: 5.6884
  Batch 864, Loss: 5.6803
  Batch 896, Loss: 5.5872
  Batch 928, Loss: 5.6045
  Batch 960, Loss: 5.5448
  Batch 992, Loss: 5.4936
  Batch 1024, Loss: 5.4738
  Batch 1056, Loss: 5.5111


In [30]:
def translate(model, src_tokenizer, tgt_tokenizer, sentence, max_len=50, device='mps'):
    """
    Tek bir cümleyi çevir (Greedy Decoding).

    Args:
        model: Eğitilmiş Transformer modeli
        src_tokenizer: Kaynak dil tokenizer (Türkçe)
        tgt_tokenizer: Hedef dil tokenizer (İngilizce)
        sentence: Çevrilecek cümle
        max_len: Maksimum çıktı uzunluğu
        device: 'cpu' veya 'mps'

    Returns:
        Çevrilmiş cümle
    """
    model.eval()  # Evaluation mode (dropout kapalı)

    with torch.no_grad():
        # 1️⃣ Kaynak cümleyi tokenize et
        src_ids = src_tokenizer.encode(sentence)  # [BOS, ..., EOS]
        src_tensor = torch.tensor([src_ids]).to(device)  # (1, src_len)

        print(f"📥 Girdi: {sentence}")
        print(f"   Tokenlar: {src_tokenizer.encode(sentence)}")
        print(f"   IDs: {src_ids}")

        # 2️⃣ Encoder'dan geçir
        src_mask = model.make_src_mask(src_tensor)
        encoder_output = model.encode(src_tensor, src_mask)

        # 3️⃣ Decoder başlangıcı: sadece <BOS>
        tgt_ids = [tgt_tokenizer.bos_token_id]

        # 4️⃣ Autoregressive üretim
        for step in range(max_len):
            tgt_tensor = torch.tensor([tgt_ids]).to(device)  # (1, current_len)
            tgt_mask = model.make_tgt_mask(tgt_tensor)

            # Decoder çıktısı
            logits = model.decode(tgt_tensor, encoder_output, src_mask, tgt_mask)

            # Son pozisyonun olasılıkları
            next_token_logits = logits[0, -1, :]  # (vocab_size,)

            # En yüksek olasılıklı token (greedy)
            next_token_id = next_token_logits.argmax().item()

            # Token'ı ekle
            tgt_ids.append(next_token_id)

            # <EOS> geldiyse dur
            if next_token_id == tgt_tokenizer.eos_token_id:
                break

        # 5️⃣ Decode et
        translation = tgt_tokenizer.decode(tgt_ids)

        print(f"📤 Çıktı: {translation}")
        print(f"   IDs: {tgt_ids}")

    return translation


In [31]:
sentence ="How to care about our health?"
translation = translate(model,en_tokenizer,tr_tokenizer,sentence,max_len=max_seq_len,device=device)
print(f"Çeviri: {translation}")

📥 Girdi: How to care about our health?
   Tokenlar: [2, 273, 35, 680, 298, 367, 408, 793, 3]
   IDs: [2, 273, 35, 680, 298, 367, 408, 793, 3]
📤 Çıktı: sağlıkmızla nasıl bakılacağınız?
   IDs: [2, 575, 757, 184, 4, 485, 427, 742, 772, 85, 595, 795, 3]
Çeviri: sağlıkmızla nasıl bakılacağınız?


In [40]:
torch.save(model.state_dict(), 'model/final_model.pth')
print(f'Model kaydedildi: model/final_model.pth')

Model kaydedildi: model/final_model.pth


### loading saved model

In [33]:
model_path = 'model/final_model.pth'

saved_model = Transformer(
    src_vocab_size=vocab_size,
    tgt_vocab_size=vocab_size,
    d_model=d_model,
    num_heads=num_heads,
    num_encoder_layers=num_encoder_layers,
    num_decoder_layers=num_decoder_layers,
    d_ff=d_ff,
    max_seq_len=max_seq_len,
    dropout=0.1,
    padd_idx=0
).to(device)

# Load state dictionary
state_dict = torch.load(model_path, map_location=device)

# Filter out unexpected keys
model_state_dict = saved_model.state_dict()
filtered_state_dict = {k: v for k, v in state_dict.items() if k in model_state_dict}
model_state_dict.update(filtered_state_dict)
saved_model.load_state_dict(model_state_dict)
saved_model.to(device)
saved_model.eval()

Transformer(
  (encoder): Encoder(
    (token_embedding): TokenEmbedding(
      (embedding): Embedding(800, 256)
    )
    (positional_encoding): PositionalEncoding(
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (layers): ModuleList(
      (0-5): 6 x EncoderLayer(
        (self_attention): MultiHeadAttention(
          (W_q): Linear(in_features=256, out_features=256, bias=True)
          (W_k): Linear(in_features=256, out_features=256, bias=True)
          (W_v): Linear(in_features=256, out_features=256, bias=True)
          (W_o): Linear(in_features=256, out_features=256, bias=True)
          (attention): ScaledDotProductAttention(
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (feed_forward): PositionwiseFeedForward(
          (linear1): Linear(in_features=256, out_features=2048, bias=True)
          (linear2): Linear

In [34]:
sentence ="But how can I prove it?"
translation = translate(saved_model,en_tokenizer,tr_tokenizer,sentence,max_len=max_seq_len,device=device)
print(f"Çeviri: {translation}")

📥 Girdi: But how can I prove it?
   Tokenlar: [2, 260, 273, 157, 570, 76, 52, 84, 793, 3]
   IDs: [2, 260, 273, 157, 570, 76, 52, 84, 793, 3]
📤 Çıktı: ama bunu nasıl kanıtlayabilirim?
   IDs: [2, 661, 7, 172, 485, 12, 296, 756, 4, 48, 143, 54, 795, 3]
Çeviri: ama bunu nasıl kanıtlayabilirim?


In [35]:
sentence ="Good morning everyone!"
translation = translate(saved_model,en_tokenizer,tr_tokenizer,sentence,max_len=max_seq_len,device=device)
print(f"Çeviri: {translation}")

📥 Girdi: Good morning everyone!
   Tokenlar: [2, 241, 128, 29, 23, 714, 676, 323, 1, 3]
   IDs: [2, 241, 128, 29, 23, 714, 676, 323, 1, 3]
📤 Çıktı: İyi sabah herkes için iyi sabah ⁇ 
   IDs: [2, 174, 759, 749, 195, 762, 65, 218, 753, 40, 57, 469, 195, 762, 65, 1, 3]
Çeviri: İyi sabah herkes için iyi sabah ⁇ 


In [38]:
sentence ="What is your name?"
translation = translate(saved_model,en_tokenizer,tr_tokenizer,sentence,max_len=max_seq_len,device=device)
print(f"Çeviri: {translation}")

📥 Girdi: What is your name?
   Tokenlar: [2, 350, 56, 261, 49, 414, 793, 3]
   IDs: [2, 350, 56, 261, 49, 414, 793, 3]
📤 Çıktı: adınız nedir?
   IDs: [2, 651, 595, 69, 459, 795, 3]
Çeviri: adınız nedir?


In [39]:
sentence ="One more example sentence for translation."
translation = translate(saved_model,en_tokenizer,tr_tokenizer,sentence,max_len=max_seq_len,device=device)
print(f"Çeviri: {translation}")

📥 Girdi: One more example sentence for translation.
   Tokenlar: [2, 253, 222, 109, 522, 42, 11, 43, 252, 68, 475, 759, 57, 772, 3]
   IDs: [2, 253, 222, 109, 522, 42, 11, 43, 252, 68, 475, 759, 57, 772, 3]
📤 Çıktı: çeviri için bir daha fazla örnek cümlesi.
   IDs: [2, 605, 8, 749, 57, 28, 129, 283, 650, 24, 122, 124, 752, 142, 773, 3]
Çeviri: çeviri için bir daha fazla örnek cümlesi.
